# 견고한 추론
- 작성자: 고려대학교 경제학과 한치록 교수, 데이터사이언스팀 이창훈 과장

[Statsmodels][sm] 패키지를 이용하여 [이분산에 견고한 표준오차](https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors), [클러스터 구조에 견고한 표준오차](https://en.wikipedia.org/wiki/Clustered_standard_errors), [Newey-West 표준오차](https://en.wikipedia.org/wiki/Newey–West_estimator)를 구하는 방법을 살펴본다.

[sm]: https://www.statsmodels.org/

## 표준오차 계산 방법을 지정하는 단계
계량경제학에서 추정값 계산 단계와 표준오차 계산 단계는 서로 구분되는 별개의 과정이다. OLS 추정량에 대하여 통상적인(ordinary) 표준오차를 구할 수도 있고 견고한 표준오차를 구할 수도 있다. 그래서 [R]에서는 OLS 추정이 완료된 다음, 주어진 OLS 추정 결과에 대하여 다양한 방식으로 표준오차를 구할 수 있다.

```r
# R
library(lmtest)
library(sandwich)
ols <- lm(y~x1+x2, data=DF)  # OLS
coeftest(ols)                # ordinary standard errors
coeftest(ols, vcov = vcovHC) # HC3 standard errors
coeftest(ols, vcov = vcovCL, cluster = ~id) # clustered standard errors
```

[Stata]는 추정과 표준오차 계산을 하나의 명령에 통합하되, 표준오차 계산 방법을 바꾸고자 하면 명령어의 옵션을 조정한다.

```s
/* Stata */
reg y x1 x2             /* OLS with ordinary standard errors */
reg y x1 x2, vce(r)     /* OLS with HC standard errors */
reg y x1 x2, vce(cl id) /* OLS with clustered standard errors */
```

[Statsmodels][sm]에서는 `fit()` 단계에서 표준오차 계산 방식을 지정할 수 있다.

```python
# Python Statsmodels
import statsmodels.formula.api as smf
model = smf.ols('y~x1+x2', data=df)  # "OLS model" (to be fitted later)
reg = model.fit()
reg.summary()                    # Show stats with ordinary standard errors
reg = model.fit(cov_type='HC3')  # To show HC3 standard errors by summary()
reg.summary()                    # Show stats with HC standard errors
```

[Statsmodels][sm]에서 표준오차 방식 지정에 관한 자세한 사항은 [해당 Statsmodels 도움말](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.get_robustcov_results.html#statsmodels.regression.linear_model.RegressionResults.get_robustcov_results)을 참조하라.

[R]: https://r-project.org/
[Stata]: https://stata.com/
[sm]: https://www.statsmodels.org/

## 이분산에 견고한 표준오차

횡단면 자료를 분석할 때에는 관측치 수가 적지만 않으면 [이분산에 견고한 표준오차](https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors)를 사용하는 것이 좋다. 이분산에 견고한 표준오차는 [statsmodels][sm]에 구현되어 있다. `fit` 메쏘드 호출 시 적절한 `cov_type` 인자를 이용한다. 아래 예를 보라.

[R]: https://r-project.org/
[Stata]: https://stata.com/
[sm]: https://www.statsmodels.org/

In [1]:
import statsmodels.formula.api as smf
import bok_da as bd
import pandas as pd

df = pd.read_csv('../data/wage1.csv')
ols_model = smf.ols('lwage ~ educ + exper + tenure', data=df)
# ols_with_ordinary_se = ols_model.fit()
ols_hc1 = ols_model.fit(cov_type='HC1')
print(ols_hc1.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:                  lwage   R-squared:                      0.3160
Model:                            OLS   Adj. R-squared:                 0.3121
No. Observations:                 526   F-statistic:                     67.76
Covariance Type:                  HC1   Prob (F-statistic):             0.0000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     0.28436    0.11171       2.55      0.011     0.06542     0.50330
educ          0.09203    0.00792      11.62      0.000     0.07650     0.10755
exper         0.00412    0.00175       2.36      0.018     0.00070     0.00754
tenure        0.02207    0.00378       5.83      0.000     0.01465     0.02948

Notes:
1. Standard Errors are heteroscedasticity robust (HC1)


하단 Notes \[1\]을 보면 표준오차가 HC1임이 표시되어 있다.

[Statsmodels][sm]에서 구하는 이분산에 견고한 표준오차 유형으로 HC0, HC1, HC2, HC3이 있다. [R]에서는 `vcovHC()`에서 어느 유형을 사용할지 지정할 수 있으며, 옵션을 주지 않으면 default로 HC3이 계산된다. [Stata]에서 `vce(r)` 옵션을 주면 HC1이 계산된다.
```
. use wage1, clear

. reg lwage educ exper tenure, vce(r)

Linear regression                               Number of obs     =        526
                                                F(3, 522)         =      67.76
                                                Prob > F          =     0.0000
                                                R-squared         =     0.3160
                                                Root MSE          =     .44086

------------------------------------------------------------------------------
             |               Robust
       lwage | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        educ |    .092029   .0079212    11.62   0.000     .0764676    .1075903
       exper |   .0041211   .0017459     2.36   0.019     .0006913    .0075509
      tenure |   .0220672    .003782     5.83   0.000     .0146374    .0294971
       _cons |   .2843595   .1117069     2.55   0.011     .0649092    .5038098
------------------------------------------------------------------------------
```

[R]: https://r-project.org/
[Stata]: https://stata.com/
[sm]: https://www.statsmodels.org/

In [2]:
ols_hc1.HC1_se

Intercept    0.111707
educ         0.007921
exper        0.001746
tenure       0.003782
dtype: float64

Stata에서 `vce(r)`이 리포트하는 표준오차가 Statsmodels의 `HC1_se`와 동일함을 확인할 수 있다.

## 클러스터 표준오차

- 패널데이터 분석 등에서 [클러스터 표준오차](https://en.wikipedia.org/wiki/Clustered_standard_errors)를 사용하기 위해서는 `cov_type`를 `"cluster"`로 지정하고 `cov_kwds`에 `"groups"`를 지정한다(아래 예 참조).  

- 아래 예에서 `C(year)`는 `year`변수를 더미변수로 처리해 각 연도별 고정효과를 회귀식에 포함하라는 의미이다. 

- `cov_kwds={'groups':df["region"]}`는 `cov_type="cluster"` 방식으로 표준오차를 계산할 때 어떤 기준으로 군집(cluster)을 나눌지 지정하는 옵션이다. 아래 예에서 `region`값이 동일한 관측치들끼리 하나의 군집이 되고 군집 내 상관관계를 허용한 표준오차를 계산하게 된다. 결과적으로 지역별 군집화를 통해 같은 지역 내 관측치들끼리의 상관성을 감안한 보다 견고헌(robust) 신뢰구간과 p-값이 산출된다.

In [3]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_csv('../data/Death.csv') # 3 year panel data
model = smf.ols('deathrate~drink+smoke+aged+vehipc+C(year)', data=df) # C: categorical
ols_cc = model.fit(cov_type="cluster", cov_kwds={'groups':df["region"]})
print(ols_cc.summary(slim=True))

                            OLS Regression Results                            
Dep. Variable:              deathrate   R-squared:                      0.9209
Model:                            OLS   Adj. R-squared:                 0.9190
No. Observations:                 258   F-statistic:                     493.2
Covariance Type:              cluster   Prob (F-statistic):             0.0000
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -0.22413    1.01916      -0.22      0.826    -2.22165     1.77340
C(year)[T.2009]   -0.37876    0.07655      -4.95      0.000    -0.52879    -0.22873
C(year)[T.2010]   -0.35100    0.09895      -3.55      0.000    -0.54493    -0.15706
drink              0.00639    0.01410       0.45      0.650    -0.02125     0.03404
smoke              0.03328    0.01939       1.72      0.086    -0.00473     0.07128
aged             

**R 결과와 비교**하면 소수점 아래 셋째 자리까지 똑같음을 알 수 있다.

```r
DF <- read.csv('../data/Death.csv')
ols <- lm(deathrate~drink+smoke+aged+vehipc+factor(year), data=DF)
coeftest(ols, vcov=vcovCL, cluster=~region)

# t test of coefficients:
# 
#                    Estimate Std. Error t value    Pr(>|t|)    
# (Intercept)      -0.2241278  1.0191647 -0.2199   0.8261175    
# drink             0.0063935  0.0141033  0.4533   0.6506992    
# smoke             0.0332761  0.0193888  1.7163   0.0873495 .  
# aged              0.4026956  0.0135248 29.7745   < 2.2e-16 ***
# vehipc            1.4079470  1.7245757  0.8164   0.4150444    
# factor(year)2009 -0.3787601  0.0765451 -4.9482 0.000001374 ***
# factor(year)2010 -0.3509959  0.0989481 -3.5473   0.0004644 ***
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
```

더 상세한 수치를 비교해 보면 정확히 일치함을 알 수 있다.

In [4]:
ols_cc.bse

Intercept          1.019165
C(year)[T.2009]    0.076545
C(year)[T.2010]    0.098948
drink              0.014103
smoke              0.019389
aged               0.013525
vehipc             1.724576
dtype: float64

**Stata 결과와 비교**해도 똑같다.

```
. import delimited using ../data/Death.csv, delim(",") clear
(encoding automatically selected: ISO-8859-1)
(9 vars, 258 obs)

. reg deathrate drink smoke aged vehipc i.year, vce(cl region)

Linear regression                               Number of obs     =        258
                                                F(6, 85)          =     493.15
                                                Prob > F          =     0.0000
                                                R-squared         =     0.9209
                                                Root MSE          =     .61396

                                (Std. err. adjusted for 86 clusters in region)
------------------------------------------------------------------------------
             |               Robust
   deathrate | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       drink |   .0063935   .0141033     0.45   0.651    -.0216475    .0344346
       smoke |   .0332761   .0193888     1.72   0.090     -.005274    .0718263
        aged |   .4026956   .0135248    29.77   0.000     .3758046    .4295866
      vehipc |   1.407947   1.724575     0.82   0.417    -2.020971    4.836865
             |
        year |
       2009  |  -.3787601   .0765451    -4.95   0.000    -.5309523    -.226568
       2010  |  -.3509959   .0989481    -3.55   0.001    -.5477312   -.1542606
             |
       _cons |  -.2241278   1.019165    -0.22   0.826      -2.2505    1.802244
------------------------------------------------------------------------------
```

이상은 자료집합에 결측치가 없는 경우에 해당한다. 만약 결측치가 있으면 다음 오류가 발생할 수 있다.

```
ValueError: The weights and list don't have the same length.
```

이 문제를 해결하기 위해서는 [`pandas.DataFrame.dropna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html)를 이용하여 미리 결측치들을 제거하고 시작해야 한다. smf.ols는 종속변수·설명변수만 보고 결측치가 있는 행을 자동으로 제거한다. 하지만 클러스터 표준오차의 그룹 변수(df["region"])는 결측치가 있는 행을 자동으로 잘라내지 않고 원본 길이를 그대로 유지하기 때문에, 양쪽 길이가 안 맞아 오류가 난다. 이 경우 아래와 같이 dropna()의 subset인자를 사용해서 결측치가 있는 행을 제거한 후 추정하면 된다.

```python
# 회귀식에 쓰이는 변수들과 클러스터 변수 이름을 모두 모아서
needed = ["deathrate", "drink", "smoke", "aged", "vehipc", "year", "region"]

# 이 중 하나라도 NaN인 행은 dropna()로 완전히 삭제
df_clean = df.dropna(subset=needed)

# 이제 그룹 길이도 딱 맞으니 오류 없이 돌릴 수 있음
model  = smf.ols('deathrate ~ drink + smoke + aged + vehipc + C(year)', data=df_clean)
ols_cc = model.fit(cov_type="cluster", cov_kwds={'groups': df_clean["region"]})
```

## Newey-West 표준오차
Newey-West 표준오차 사용을 위해서는 `cov_type` 옵션으로 "HAC"를 사용한다. 자세한 설명은 [여기](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.get_robustcov_results.html#statsmodels.regression.linear_model.RegressionResults.get_robustcov_results) 참조. 다음 예는 [R sandwich 패키지의 NeweyWest 도움말](https://sandwich.r-forge.r-project.org/reference/NeweyWest.html)을 참조하였다.

In [6]:
import pandas as pd
import statsmodels.formula.api as smf

df = pd.read_stata('../data/idle2.dta')
model = smf.ols('usr ~ idle', data=df)
ols_hac = model.fit(cov_type = "HAC", cov_kwds={'maxlags':3, 'use_correction':True})
print(ols_hac.summary())

                            OLS Regression Results                            
Dep. Variable:                    usr   R-squared:                      0.5006
Model:                            OLS   Adj. R-squared:                 0.4828
Method:                 Least Squares   F-statistic:                     10.90
Date:                Thu, 22 May 2025   Prob (F-statistic):             0.0026
Time:                        14:36:40   Log-Likelihood:                -71.743
No. Observations:                  30   AIC:                             147.5
Df Residuals:                      28   BIC:                             150.3
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    23.13483    6.32703       3.66      0.0

이 결과는 **R로써 다음과 같이 복원** 가능하다.

```r
# R codes
library(lmtest)
library(sandwich)
DF <- readstata13::read.dta13('idle2.dta')
ols <- lm(usr~idle, data=DF)
coeftest(ols, vcov=vcovHAC, lag=3, prewhite = FALSE, adjust = TRUE)

# t test of coefficients:
# 
#             Estimate Std. Error t value  Pr(>|t|)    
# (Intercept) 23.134828   6.327031  3.6565 0.001047 **
# idle        -0.228150   0.069093 -3.3021 0.002626 **
# ---
# Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
```

**Stata에서도 복원** 가능하다.

```
. webuse idle2, clear

. tsset time

Time variable: time, 1 to 30
        Delta: 1 unit

. newey usr idle, lag(3)

Regression with Newey–West standard errors      Number of obs     =         30
Maximum lag = 3                                 F(  1,        28) =      10.90
                                                Prob > F          =     0.0026

------------------------------------------------------------------------------
             |             Newey–West
         usr | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
        idle |  -.2281501   .0690927    -3.30   0.003    -.3696801     -.08662
       _cons |   23.13483   6.327031     3.66   0.001     10.17449    36.09516
------------------------------------------------------------------------------
```